In [2]:
import pandas as pd
df = pd.read_csv("/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
print(df.shape)
df.head()

(225745, 79)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
!pip install boruta

In [5]:
print(df.columns)

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [6]:
X = df.drop(columns=[' Label'])
y = df[' Label']

from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

X = X.fillna(0)

In [7]:
import numpy as np

# Replace inf and -inf with NaN
X = X.replace([np.inf, -np.inf], np.nan)

# Fill NaN with 0 (or column mean if you prefer)
X = X.fillna(0)

# Also clip very large values to a reasonable range
X = np.clip(X, -1e10, 1e10)


In [8]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

boruta_selector = BorutaPy(
    rf, 
    n_estimators='auto', 
    verbose=2, 
    random_state=42
)

boruta_selector.fit(X.values, y)

selected_features = X.columns[boruta_selector.support_].to_list()
# Save to CSV for reuse
pd.Series(selected_features).to_csv("selected_features.csv", index=False)

print("Selected Features saved!")

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	60
Tentative: 	6
Rejected: 	12
Iteration: 	9 / 100
Confirmed: 	60
Tentative: 	6
Rejected: 	12
Iteration: 	10 / 100
Confirmed: 	60
Tentative: 	6
Rejected: 	12
Iteration: 	11 / 100
Confirmed: 	60
Tentative: 	6
Rejected: 	12
Iteration: 	12 / 100
Confirmed: 	61
Tentative: 	5
Rejected: 	12
Iteration: 	13 / 100
Confirmed: 	61
Tentative: 	5
Rejected: 	12
Iteration: 	14 / 100
Confirmed: 	61
Tentative: 	5
Rejected: 	12
Iteration: 	15 / 100
Confirmed: 	61
Tentative: 	5
Rejected: 	12
Iteration: 	16 / 100
Confirmed: 	61
Tentative: 	5
Reject

In [ ]:
# Reload saved features
selected_features = pd.read_csv("selected_features.csv", header=None)[0].tolist()
X_selected = df[selected_features]

In [9]:
X_selected = X[selected_features]

print("Shape before:", X.shape)
print("Shape after:", X_selected.shape)

Shape before: (225745, 78)
Shape after: (225745, 62)


In [10]:

from sklearn.preprocessing import StandardScaler, LabelEncoder
# Scale features for NN models
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Encode labels (0,1,... for classes)
y = LabelEncoder().fit_transform(y)

print("Final dataset shape:", X_scaled.shape, y.shape)


Final dataset shape: (225745, 62) (225745,)


In [11]:
from sklearn.model_selection import train_test_split

# Reduce to 50k samples for faster experiments
X_small, _, y_small, _ = train_test_split(X_scaled, y, train_size=50000, stratify=y, random_state=42)

# Split train/val/test
X_train_full, X_test, y_train_full, y_test = train_test_split(X_small, y_small, test_size=0.2, stratify=y_small, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (32000, 62) Val: (8000, 62) Test: (10000, 62)


In [12]:
from sklearn.model_selection import train_test_split

# First split into train+val and test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Now split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (144476, 62) Val: (36120, 62) Test: (45149, 62)


In [14]:
# Required imports
import numpy as np, random, time, os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# -------------------------
# Determinism helper
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------------------------
# Focal Loss
# -------------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, inputs, targets):
        # inputs: logits (batch, C); targets: (batch,)
        ce = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce)
        loss = self.alpha * (1 - pt) ** self.gamma * ce
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

# -------------------------
# Encoders (return embedding vector)
# -------------------------
class CNNEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim=128):
        super().__init__()
        # we'll treat features as a 1D "sequence" of length=input_dim with 1 channel
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(128, emb_dim)
    def forward(self, x):
        # x: [batch, input_dim]
        x = x.unsqueeze(1)                # [batch, 1, input_dim]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)      # [batch, 128]
        return self.fc(x)                 # [batch, emb_dim]

class GRUEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim=128):
        super().__init__()
        # interpret each sample as sequence length = input_dim, features=1
        self.gru = nn.GRU(input_size=1, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, emb_dim)
    def forward(self, x):
        # x: [batch, input_dim]
        x = x.unsqueeze(-1)               # [batch, seq=input_dim, 1]
        _, h = self.gru(x)                # h: [1, batch, 128]
        h = h.squeeze(0)
        return self.fc(h)                 # [batch, emb_dim]

class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim=128, nheads=4, nlayers=2):
        super().__init__()
        # embed each feature into a d_model vector
        d_model = 128
        self.embed = nn.Linear(1, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nheads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=nlayers)
        self.pool_fc = nn.Linear(d_model, emb_dim)
    def forward(self, x):
        # x: [batch, input_dim]
        x = x.unsqueeze(-1)               # [batch, seq=input_dim, 1]
        x = self.embed(x)                 # [batch, seq, d_model]
        x = self.transformer(x)           # [batch, seq, d_model]
        x = x.mean(dim=1)                 # global average pool over seq
        return self.pool_fc(x)            # [batch, emb_dim]

# -------------------------
# Combined model builder
# -------------------------
class CombinedModel(nn.Module):
    def __init__(self, encoders, emb_dim=128, num_classes=2, dropout=0.3):
        """
        encoders: list of encoder modules (nn.Module). Each encoder outputs an embedding vector.
        """
        super().__init__()
        self.encoders = nn.ModuleList(encoders)
        total_emb = emb_dim * len(encoders)
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(total_emb, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        embs = [enc(x) for enc in self.encoders]  # each [batch, emb_dim]
        concat = torch.cat(embs, dim=1)           # [batch, total_emb]
        return self.head(concat)                  # logits

# -------------------------
# Training & evaluation utilities
# -------------------------
def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    epoch_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)
    return epoch_loss / len(loader.dataset)

def eval_model(model, loader, device):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            logits = model(xb)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            trues.extend(yb.numpy())
    return np.array(preds), np.array(trues)

def compute_metrics(y_true, y_pred):
    out = {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall_macro': recall_score(y_true, y_pred, average='macro', zero_division=0)
    }
    return out

# -------------------------
# Experiment runner (runs combos)
# -------------------------
def run_experiment_combo(X_train, y_train, X_val, y_val, X_test, y_test,
                         combo_name, encoder_classes, input_dim, num_classes,
                         seed=42, device='cuda' if torch.cuda.is_available() else 'cpu',
                         epochs=20, batch_size=256, lr=1e-3, emb_dim=128):
    set_seed(seed)
    device = torch.device(device)
    # instantiate encoders
    encs = [cls(input_dim=input_dim, emb_dim=emb_dim) for cls in encoder_classes]
    model = CombinedModel(encs, emb_dim=emb_dim, num_classes=num_classes).to(device)
    loss_fn = FocalLoss(alpha=1.0, gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # DataLoaders
    tr_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                        torch.tensor(y_train, dtype=torch.long)),
                           batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                         torch.tensor(y_val, dtype=torch.long)),
                           batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                          torch.tensor(y_test, dtype=torch.long)),
                            batch_size=batch_size, shuffle=False)

    best_val_f1 = -1.0
    best_state = None
    history = []
    start = time.time()
    for ep in range(epochs):
        tr_loss = train_one_epoch(model, tr_loader, optimizer, loss_fn, device)
        preds_val, trues_val = eval_model(model, val_loader, device)
        m = compute_metrics(trues_val, preds_val)
        history.append({'epoch': ep+1, 'train_loss': tr_loss, **m})
        if m['f1_macro'] > best_val_f1:
            best_val_f1 = m['f1_macro']
            best_state = {k: v.cpu().clone() for k,v in model.state_dict().items()}

    # load best and eval on test
    model.load_state_dict(best_state)
    preds_test, trues_test = eval_model(model, test_loader, device)
    test_metrics = compute_metrics(trues_test, preds_test)
    elapsed = time.time() - start

    result = {
        'combo': combo_name,
        'seed': seed,
        'train_samples': len(X_train),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'epochs_trained': epochs,
        'time_s': elapsed,
        **test_metrics
    }
    return result, history

# -------------------------
# How to run all combos
# -------------------------
def run_all_combinations(X_train, y_train, X_val, y_val, X_test, y_test, input_dim, num_classes,
                         seeds=[42], out_csv='ablation_results.csv', combos_to_run=None):
    all_combos = {
        'CNN': [CNNEncoder],
        'GRU': [GRUEncoder],
        'Transformer': [TransformerEncoder],
        'CNN+GRU': [CNNEncoder, GRUEncoder],
        'CNN+Transformer': [CNNEncoder, TransformerEncoder],
        'GRU+Transformer': [GRUEncoder, TransformerEncoder],
        'CNN+GRU+Transformer': [CNNEncoder, GRUEncoder, TransformerEncoder]
    }
    if combos_to_run is None:
        combos = all_combos
    else:
        combos = {k: all_combos[k] for k in combos_to_run if k in all_combos}
    
    results = []
    for seed in seeds:
        for name, enc_list in combos.items():
            print(f"Running {name} (seed {seed}) ...")
            res, hist = run_experiment_combo(
                X_train, y_train, X_val, y_val, X_test, y_test,
                combo_name=name,
                encoder_classes=enc_list,
                input_dim=input_dim,
                num_classes=num_classes,
                seed=seed
            )
            results.append(res)
            pd.DataFrame(hist).to_csv(f'history_{name}_seed{seed}.csv', index=False)
            # append results incrementally
            pd.DataFrame(results).to_csv(out_csv, mode='a', header=not os.path.exists(out_csv), index=False)
    return pd.DataFrame(results)


# -------------------------
# Example usage (assuming you have X and y prepared)
# -------------------------
# 1) Prepare data splits (stratify)
# X_scaled and y (numpy arrays)
# X_train_full, X_test, y_train_full, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42)

# 2) Run
# df_results = run_all_combinations(X_train, y_train, X_val, y_val, X_test, y_test,
#                                  input_dim=X_train.shape[1], num_classes=len(np.unique(y)), seeds=[42, 7, 123])
# print(df_results)
# df_results.to_csv('ablation_summary.csv', index=False)


In [ ]:
df_results_cnn = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['CNN'],
    out_csv='results_partial.csv'
)


Running CNN (seed 42) ...


In [ ]:
df_results_gru = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['GRU'],
    out_csv='results_partial.csv'
)

In [ ]:
df_results_transformer = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['Transformer'],
    out_csv='results_partial.csv'
)

In [ ]:
df_results_cnn_gru = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['CNN+GRU'],
    out_csv='results_partial.csv'
)

In [ ]:

df_results_cnn_transformer = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['CNN+Transformer'],
    out_csv='results_partial.csv'
)

In [ ]:
df_results_gru_transformer = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['GRU+Transformer'],
    out_csv='results_partial.csv'
)

In [ ]:
# Run three-model hybrid
df_results_all = run_all_combinations(
    X_train, y_train, X_val, y_val, X_test, y_test,
    input_dim=X_train.shape[1],
    num_classes=len(np.unique(y)),
    seeds=[42],
    combos_to_run=['CNN+GRU+Transformer'],
    out_csv='results_partial.csv'
)